# EEG - Flow

## 2. Bridges and auto bads channels

Determine gel bridges and save the gel bridges topography. Interpolate gel bridges if possible. Mark bad channels with the PREP pipeline.

Last edit: 20.06.2023 11:26
@anguyen

In [ ]:
import multiprocessing as mp
import time
from itertools import product
from typing import List

from mne import set_log_level as set_log_level_mne
from mne.viz import set_browser_backend

from eeg_flow import set_log_level
from eeg_flow.utils import parallel
from eeg_flow.tasks import bridges_and_autobads

In [ ]:
set_log_level_mne("WARNING")
set_log_level("INFO")

%matplotlib qt
set_browser_backend("qt")

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [ ]:
PARTICIPANTS_WITH_GROUPS: List[str] = []  # List of str "Pxx-Gy", e.g. ["P02-G2"]
TASKS: List[str] = []  # ["oddball"], ["UT"] or ["oddball", "UT"]
RUNS: List[int] = []  # [1], [2] or [1, 2]

inputs = [
    (t[0].split("-") + list(t[1:]))
    for t in product(PARTICIPANTS_WITH_GROUPS, TASKS, RUNS)
]
print(inputs)

This step is divided in:
- Visualization of gel-bridges and rejection of the dataset if required
- Interpolation of gel-bridges
- Annotation of bad channels with the PREP pipeline

The created derivatives are:
- Plot of the gel-bridges (`_bridges.svg`)
- Raw with interpolated gel-bridged channels (`_with-bads_raw.fif`)

More information about gel bridgs in this [MNE tutorial](https://mne.tools/stable/auto_examples/preprocessing/eeg_bridging.html).

In [ ]:
%%time
current_time = time.strftime("%H:%M:%S")
print("Start time", current_time)

assert len(inputs) != 0  # sanity-check
n_jobs = min(len(inputs), mp.cpu_count() - 2)
parallel(bridges_and_autobads, n_jobs, inputs)